**Data Dictionary**


**Survival** = 0 = No, 1 = Yes


**Pclass** = Ticket class, 1 = 1st, 2 = 2nd, 3 = 3rd


**Sex**


**Age** = Age in years


**Sibsp** = # of siblings / spouses aboard the Titanic


**Parch** = # of parents / children aboard the Titanic


**Ticket** = Ticket number


**Fare** = Passenger fare


**Cabin** = Cabin number


**Embarked** = Port of Embarkation, C = Cherbourg, Q = Queenstown, S = Southampton

In [1]:
import pandas as pd
import pylab as pl
import numpy as np
from sklearn import svm

from sklearn import tree
from sklearn.tree import DecisionTreeClassifier

Train information

In [2]:
Train = pd.read_csv("train.csv")
Train

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S
...,...,...,...,...,...,...,...,...,...,...,...,...
886,887,0,2,"Montvila, Rev. Juozas",male,27.0,0,0,211536,13.0000,NaN,S
887,888,1,1,"Graham, Miss. Margaret Edith",female,19.0,0,0,112053,30.0000,B42,S
888,889,0,3,"Johnston, Miss. Catherine Helen ""Carrie""",female,NaN,1,2,W./C. 6607,23.4500,NaN,S
889,890,1,1,"Behr, Mr. Karl Howell",male,26.0,0,0,111369,30.0000,C148,C


Test information

In [3]:
Test = pd.read_csv("test.csv")
Test

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S
...,...,...,...,...,...,...,...,...,...,...,...
413,1305,3,"Spector, Mr. Woolf",male,NaN,0,0,A.5. 3236,8.0500,NaN,S
414,1306,1,"Oliva y Ocana, Dona. Fermina",female,39.0,0,0,PC 17758,108.9000,C105,C
415,1307,3,"Saether, Mr. Simon Sivertsen",male,38.5,0,0,SOTON/O.Q. 3101262,7.2500,NaN,S
416,1308,3,"Ware, Mr. Frederick",male,NaN,0,0,359309,8.0500,NaN,S


Checking the number of rows and columns per table

In [4]:
print(Train.shape, Test.shape)

(891, 12) (418, 11)


Sum missing values in each column

In [5]:
Train.isnull().sum()

PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            177
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
dtype: int64

In [6]:
Test.isnull().sum()

PassengerId      0
Pclass           0
Name             0
Sex              0
Age             86
SibSp            0
Parch            0
Ticket           0
Fare             1
Cabin          327
Embarked         0
dtype: int64

Drop Fare values 0

In [7]:
Train = Train.drop(Train.index[[806, 674, 302, 179, 633, 413, 597, 815, 822, 277, 271, 732, 466, 481, 263]])

Define the training target

In [8]:
Y_train = Train['Survived']

Create a complete table including Train and Test information

In [9]:
TT = [Train, Test]
TT = pd.concat(TT)
TT.shape

(1294, 12)

In [10]:
TT.reset_index(drop=True, inplace=True)
TT.tail()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
1289,1305,NaN,3,"Spector, Mr. Woolf",male,NaN,0,0,A.5. 3236,8.0500,NaN,S
1290,1306,NaN,1,"Oliva y Ocana, Dona. Fermina",female,39.0,0,0,PC 17758,108.9000,C105,C
1291,1307,NaN,3,"Saether, Mr. Simon Sivertsen",male,38.5,0,0,SOTON/O.Q. 3101262,7.2500,NaN,S
1292,1308,NaN,3,"Ware, Mr. Frederick",male,NaN,0,0,359309,8.0500,NaN,S
1293,1309,NaN,3,"Peter, Master. Michael J",male,NaN,1,1,2668,22.3583,NaN,C


Look for the total missing values

In [11]:
TT.isnull().sum()

PassengerId       0
Survived        418
Pclass            0
Name              0
Sex               0
Age             255
SibSp             0
Parch             0
Ticket            0
Fare              1
Cabin          1002
Embarked          2
dtype: int64

Remove unnecessary columns including Cabin because there is a lot missing quantity

In [12]:
TT.drop(['Survived', 'PassengerId', 'Cabin', 'Ticket', 'Name'], axis=1, inplace=True)
TT

,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,3,male,22.0,1,0,7.2500,S
1,1,female,38.0,1,0,71.2833,C
2,3,female,26.0,0,0,7.9250,S
3,1,female,35.0,1,0,53.1000,S
4,3,male,35.0,0,0,8.0500,S
...,...,...,...,...,...,...,...
1289,3,male,NaN,0,0,8.0500,S
1290,1,female,39.0,0,0,108.9000,C
1291,3,male,38.5,0,0,7.2500,S
1292,3,male,NaN,0,0,8.0500,S


In [13]:
TT.isnull().sum()

Pclass        0
Sex           0
Age         255
SibSp         0
Parch         0
Fare          1
Embarked      2
dtype: int64

There are 2 missing values in Embarked column, we can replace those by frequency

In [14]:
TT['Embarked'].value_counts()

S    899
C    270
Q    123
Name: Embarked, dtype: int64

In [15]:
TT['Embarked'].fillna('S', inplace=True)

In [16]:
TT.isnull().sum()

Pclass        0
Sex           0
Age         255
SibSp         0
Parch         0
Fare          1
Embarked      0
dtype: int64

There is 1 missing value in Fare column, we can replace it by mean

In [17]:
TT['Fare'].value_counts()

8.0500     60
13.0000    59
7.7500     55
26.0000    50
7.8958     49
           ..
25.7000     1
26.3875     1
7.8000      1
7.3125      1
7.1417      1
Name: Fare, Length: 281, dtype: int64

In [18]:
TT['Fare'].fillna(TT['Fare'].mean(), inplace = True)
TT['Fare'] = round(TT['Fare'])

In [19]:
TT.isnull().sum()

Pclass        0
Sex           0
Age         255
SibSp         0
Parch         0
Fare          0
Embarked      0
dtype: int64

There are several missing values in the Age column, we can rescue this column using the mean value and Standard deviation.

In [20]:
TT['Age'].value_counts()

24.00    47
22.00    43
21.00    41
30.00    40
18.00    39
         ..
76.00     1
22.50     1
74.00     1
80.00     1
0.67      1
Name: Age, Length: 98, dtype: int64

In [21]:
Mean_Age = TT['Age'].mean()
Mean_Age

29.845688161693936

In [22]:
Std_Age = TT['Age'].std()
Std_Age

14.435409234643902

In [23]:
Null_Age = TT['Age'].isnull().sum()
Null_Age

255

In [24]:
Age_Copy = TT['Age'].copy()
Age_Copy

0       22.0
1       38.0
2       26.0
3       35.0
4       35.0
        ... 
1289     NaN
1290    39.0
1291    38.5
1292     NaN
1293     NaN
Name: Age, Length: 1294, dtype: float64

In [25]:
Random_Age = np.random.randint(Mean_Age - Std_Age, Mean_Age + Std_Age, size = Null_Age)
Random_Age

array([35, 21, 20, 38, 30, 27, 15, 38, 17, 37, 23, 28, 40, 27, 38, 22, 42,
       36, 19, 40, 43, 37, 20, 22, 16, 33, 36, 17, 31, 38, 33, 33, 15, 17,
       28, 27, 23, 15, 32, 15, 43, 16, 35, 22, 21, 31, 16, 43, 41, 25, 41,
       29, 24, 24, 31, 26, 25, 22, 35, 34, 30, 17, 21, 32, 42, 25, 43, 30,
       25, 33, 43, 40, 32, 15, 18, 23, 23, 25, 38, 36, 22, 43, 26, 22, 30,
       35, 35, 17, 27, 17, 31, 38, 33, 15, 30, 40, 34, 19, 26, 20, 25, 34,
       27, 24, 41, 18, 35, 21, 42, 38, 30, 42, 28, 25, 43, 27, 34, 25, 32,
       27, 40, 41, 24, 41, 16, 39, 43, 30, 31, 18, 17, 42, 17, 37, 16, 18,
       21, 19, 19, 38, 22, 34, 25, 35, 20, 42, 35, 30, 26, 39, 24, 15, 31,
       22, 18, 19, 16, 16, 21, 25, 40, 32, 19, 42, 16, 43, 33, 43, 41, 20,
       24, 20, 33, 23, 35, 30, 33, 39, 28, 23, 16, 20, 37, 21, 27, 33, 17,
       33, 35, 23, 37, 17, 40, 33, 24, 18, 42, 36, 38, 22, 16, 41, 26, 41,
       38, 28, 38, 40, 27, 27, 36, 15, 36, 23, 42, 32, 19, 42, 23, 16, 31,
       38, 30, 28, 16, 25

In [26]:
Age_Copy[np.isnan(Age_Copy)] = Random_Age
TT['Age'] = Age_Copy
TT['Age'] = round(TT['Age'])

In [27]:
print(TT.isnull().sum())

Pclass      0
Sex         0
Age         0
SibSp       0
Parch       0
Fare        0
Embarked    0
dtype: int64


After we deal with the missing data, we need to get numerical variables for column Sex and Embarked.

In [28]:
TT

,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,3,male,22.0,1,0,7.0,S
1,1,female,38.0,1,0,71.0,C
2,3,female,26.0,0,0,8.0,S
3,1,female,35.0,1,0,53.0,S
4,3,male,35.0,0,0,8.0,S
...,...,...,...,...,...,...,...
1289,3,male,15.0,0,0,8.0,S
1290,1,female,39.0,0,0,109.0,C
1291,3,male,38.0,0,0,7.0,S
1292,3,male,27.0,0,0,8.0,S


In [29]:
TT = pd.concat([TT, pd.get_dummies(TT['Embarked'])], axis = 1)
TT = pd.concat([TT, pd.get_dummies(TT['Sex'])], axis = 1)
TT = pd.concat([TT, pd.get_dummies(TT['Pclass'])], axis = 1)
TT

,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked,C,Q,S,female,male,1,2,3
0,3,male,22.0,1,0,7.0,S,0,0,1,0,1,0,0,1
1,1,female,38.0,1,0,71.0,C,1,0,0,1,0,1,0,0
2,3,female,26.0,0,0,8.0,S,0,0,1,1,0,0,0,1
3,1,female,35.0,1,0,53.0,S,0,0,1,1,0,1,0,0
4,3,male,35.0,0,0,8.0,S,0,0,1,0,1,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1289,3,male,15.0,0,0,8.0,S,0,0,1,0,1,0,0,1
1290,1,female,39.0,0,0,109.0,C,1,0,0,1,0,1,0,0
1291,3,male,38.0,0,0,7.0,S,0,0,1,0,1,0,0,1
1292,3,male,27.0,0,0,8.0,S,0,0,1,0,1,0,0,1


In [30]:
TT.drop(['Sex', 'Embarked', 'C', 'S', 'Age', 'SibSp', 'Pclass', 'Parch', 1,2], axis=1, inplace=True)
TT

,Fare,Q,female,male,3
0,7.0,0,0,1,1
1,71.0,0,1,0,0
2,8.0,0,1,0,1
3,53.0,0,1,0,0
4,8.0,0,0,1,1
...,...,...,...,...,...
1289,8.0,0,0,1,1
1290,109.0,0,1,0,0
1291,7.0,0,0,1,1
1292,8.0,0,0,1,1


Then we define X for training and test

In [31]:
X_train = TT.iloc[:876]
X_test = TT.iloc[876:]
print(X_train.shape, X_test.shape)
print(TT.shape)

(876, 5) (418, 5)
(1294, 5)


In [32]:
Train.iloc[:,1]

0      0
1      1
2      1
3      1
4      0
      ..
886    0
887    1
888    0
889    1
890    0
Name: Survived, Length: 876, dtype: int64

Now we can predict with a model using Support Vector Machine (SVM), this helps us to build and train a model using the information of the passengers, and classify who survived.

In [33]:
Tree = DecisionTreeClassifier().fit(X_train, Y_train)
Tree_pred = Tree.predict(X_test)

In [34]:
data = Tree_pred
DF = pd.DataFrame(data)
DF

,0
0,0
1,1
2,0
3,0
4,1
...,...
413,0
414,1
415,0
416,0


Creating a CSV document to submit the results

In [35]:
Results = [Test, DF]
Results = pd.concat(Results, axis = 1)
Results.shape

(418, 12)

In [36]:
Results

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,0
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q,0
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S,1
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q,0
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S,0
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S,1
...,...,...,...,...,...,...,...,...,...,...,...,...
413,1305,3,"Spector, Mr. Woolf",male,NaN,0,0,A.5. 3236,8.0500,NaN,S,0
414,1306,1,"Oliva y Ocana, Dona. Fermina",female,39.0,0,0,PC 17758,108.9000,C105,C,1
415,1307,3,"Saether, Mr. Simon Sivertsen",male,38.5,0,0,SOTON/O.Q. 3101262,7.2500,NaN,S,0
416,1308,3,"Ware, Mr. Frederick",male,NaN,0,0,359309,8.0500,NaN,S,0


In [37]:
Results.drop(['Pclass', 'Name', 'Sex', 'Age', 'SibSp', 'Parch', 'Ticket', 'Fare', 'Cabin', 'Embarked'], axis=1, inplace=True)
Results

,PassengerId,0
0,892,0
1,893,1
2,894,0
3,895,0
4,896,1
...,...,...
413,1305,0
414,1306,1
415,1307,0
416,1308,0


In [38]:
Results.rename(columns={0:'Survived'}, inplace = True)
Results

,PassengerId,Survived
0,892,0
1,893,1
2,894,0
3,895,0
4,896,1
...,...,...
413,1305,0
414,1306,1
415,1307,0
416,1308,0


In [39]:
Results.to_excel('Titanic_predictions_SVM7.xlsx', sheet_name='SVM')

**The prediction accuracy of this model with Support Vector Machines was of 76.55% of people that survived.**